In [8]:
import os
import streamlit as st
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
os.environ["GROQ_API"] = os.getenv("GROQ_API")

groq_api=os.getenv("GROQ_API")

llm=ChatGroq(groq_api_key=groq_api,model_name='Llama3-8b-8192')
prompt=ChatPromptTemplate.from_template(
    """
    Answer the question based on the context provided.
    Please provide the most accurate answer response based on the question and context.
    <context>
    {context}
    </context>
    Question:{input}

    """ 

)

In [12]:
embeddings = OllamaEmbeddings()
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents=text_splitter.split_documents(docs[:50])
vectors=FAISS.from_documents(final_documents,embeddings)

In [13]:
vectorstore = FAISS.load_local(
    "vectors", 
    embeddings, 
    allow_dangerous_deserialization=True
)


In [ ]:
user_prompt="what is noisy data"
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vectorstore.as_retriever()
retrievel_chain=create_retrieval_chain(retriever,document_chain)
response=retrievel_chain.invoke({'input':user_prompt})
response['answer']